In [ ]:
import json
import os
import math

import utils.instrument as instr

with open('./config/scpi_commands.json') as f:
    scpi_cmds = json.load(f)['scpi_cmds']

ip_addr = r'192.168.0.1'

sa = instr.SpectrumAnalyzer(scpi_cmds, ip_addr)

Connected to: Rohde&Schwarz,FSQ-26,200750/026,4.75 successfully.


In [6]:
def unit_measurement(sa: instr.SpectrumAnalyzer, sa_config: dict):
    sa.config(param='continues_sweep', value='OFF')  # stop sweep
    input('Waiting for board launch up. Press enter to start measurement.')

    sa.config(param=sa_config)  # start continues sweep

    input(f"Waiting trace to stabilize. Press Enter to stop sweep.")
    sa.config(param='continues_sweep', value='OFF')  # stop sweep

    peak_freq, peak_power = sa.aquire_peak_point()

    return peak_freq, peak_power

In [ ]:
import utils.components as comps
import utils.instrument as instr

t_cal = comps.TraceDataCalculator()

def measure_obw_and_sbw(sa: instr.SpectrumAnalyzer, giteki_dict: dict, rule='49_27_3'):
    # calculate center freq. and span
    start_freq = float(giteki_dict['masks'][rule]['obw'].split('~')[0])  #GHz
    stop_freq = float(giteki_dict['masks'][rule]['obw'].split('~')[1])   #GHz
    sbw_limit = float(giteki_dict['masks']['sbw']) * 1e3  # MHz
    obw_limit = (stop_freq - start_freq) * 1e3  # MHz
    center_freq = (start_freq + stop_freq) / 2  # GHz
    
    # place tmp maker for obw measurement
    sa.config(param='obw_percent', value='99PCT')
    sa.config(param='obw_measure', value='SEL OBW', delimiter=':')
    sa.config(param='obw_measure', value='RES? OBW', delimiter=':')

    # construct other cfg of spectrum analyzer
    sa_config = {'center_freq': f"{center_freq}GHz", 'span': f"{obw_limit * 3}MHz"}
    sa_config.update(giteki_dict['obw_and_sbw'])

    unit_measurement(sa, sa_config)          # perform measurement
    sa.save_screenshot('obw_and_sbw.png')    # save screenshot
    sa.save_trace_to_csv('obw_and_sbw.csv')  # save trace data

    # use trace data to calculate obw and sbw
    global t_cal
    t_cal.read_trace_data('obw_and_sbw.csv')

    # calculate obw
    obw, l_freq, r_freq = t_cal.calculate_obw()
    obw_status = "Passed" if obw <= obw_limit else "Failed"
    l_freq_status = "Passed" if l_freq >= start_freq else "Failed"
    r_freq_status = "Passed" if r_freq <= stop_freq else "Failed"
    # calculate abw
    sbw = t_cal.calculate_sbw()
    sbw_status = "Passed" if sbw >= sbw_limit else "Failed"

    return {
        'obw': [f"{obw}MHz", obw_status],
        'obw lower freq': [f"{l_freq}GHz", l_freq_status],
        'obw upper freq': [f"{r_freq}GHz", r_freq_status],
        'sbw': [f"{sbw}MHz", sbw_status]
    }
